Run the cells in order to generate a quantum circuit and download the sonification as a MIDI file! The circuit is defined in the 3rd cell (feel free to play with it!) and the sonification happens in the 4th cell.

In [ ]:
# Install dependencies
!pip install MIDIUtil
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for MIDIUtil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54567 sha256=8031e39775c32052bd841b17d4fa4033510cd4631fe84788604e423886c5769c
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built MIDIUtil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5

In [ ]:
# helper functions for getting the statevector at each step in a quantum circuit, and the probabilities of each state for each of those statevectors

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# go instruction-by-instruction in the given quantum circuit and get the statevector after each
def get_statevector_at_steps(qc):

  # list to hold the statevectors
  statevectors = []

  # create a new quantum circuit to apply the instructions to get intermediate results
  nqc = QuantumCircuit(qc.num_qubits, qc.num_clbits)
  nqc.clear();

  # go through all the instructions in the provided quantum circuit
  instructions = qc.data
  for instruction in instructions:
    nqc.append(instruction) # apply them to the new quantum circuit
    statevectors.append(Statevector.from_instruction(nqc)) # append the current statevector of the new circuit to the statevectors list

  return statevectors

def get_probabilities_at_steps(qc):
  return [sv.probabilities() for sv in get_statevector_at_steps(qc)]

In [ ]:
# define a quantum circuit and get the probabilities

from qiskit.circuit.library import C3XGate
from qiskit import QuantumCircuit
c3x = C3XGate()

# create a new quantum circuit
# edit this to define your own!
qc = QuantumCircuit(4)
qc.cx(0, 1)
qc.h(1)
qc.cx(0,2)
qc.h(1)
qc.x(2)
qc.append(c3x, [0, 1, 2, 3])

probability_distributions = get_probabilities_at_steps(qc)

In [ ]:
# run this block to generate and download the MIDI file

#Each QuBit state is mapped to a different pitch
#|00> = C3 (MIDI Number 48)
#|01> = G3 (MIDI Number 55)
#|10> = E4 (MIDI Number 64)
#|11> = Bb4 (MIDI Number 70)

#This way, each musical layer is not only present in a different musical space, but also results in a unique musical interval.
#Probabilities for each state are calculated and mapped linearly to rhythmic energy. Higher probability = faster rhythm. (And probability twice as high corresponds to a rhythm twice as fast)

import numpy as np
from google.colab import files
from midiutil import MIDIFile

#Calculating # of states
num_states = 2**qc.num_qubits

#Excluding probabilities equal to zero
#Also excluding very small probabilities
Revised_probability_distributions = [];
for probability_distribution in probability_distributions:
  All_Nonzero_Probability_States = []
  for i in range(0,num_states):
    if probability_distribution[i] < 1/64:  #Setting the threshold at 1/64 will mean that no note values smaller than 64th notes will be present in the final MIDI result
      probability_distribution[i] = 0
    if probability_distribution[i] != 0:
      All_Nonzero_Probability_States.append(probability_distribution[i])
  Revised_probability_distributions.append(All_Nonzero_Probability_States)


#Determining ratio of probabilities to the minimum value
Set_of_probability_ratios = []
for i in range(0,len(qc.data)):
  #calculating minimum (nonzero) probability for each step in the circuit
  Minimum_nonzero_probability = np.min(Revised_probability_distributions[i])
  #determining ratio of all other proabilities (for that step in the circuit) to the minimum nonzero prob.
  probability_ratios = []
  for j in range(0,num_states):
    probability_ratios.append(probability_distributions[i][j]/Minimum_nonzero_probability)
  #Set_of_probability_ratios is an array of arrays (each entry in the outer array is an array of prob. ratios for each step in the circuit, # entries in outer array = # of instructions in circuit)
  Set_of_probability_ratios.append(probability_ratios)

print(Set_of_probability_ratios)
#Assigning results to MIDI Data

#Creating a MIDI File with 4 tracks (separate track for each state, which allows us to assign a different instrument to each state if we like)
Qubit_Superposition_Sonification_MIDI = MIDIFile(numTracks = num_states)
Qubit_Superposition_Sonification_MIDI.addTempo(track = 0, time=0, tempo=60)

#The following code generates one 4/4 measure of sonification FOR EACH STEP IN THE CIRCUIT, with the minimum probability value assigned to the quarter note
Duration_in_quarter_notes = 4;

#List_of_MIDI_Notes = [48, 55, 64, 70] #MIDI Notes corresponding to |00>, |01>, |10>, and |11> states
List_of_MIDI_Notes = np.random.randint(40,80,num_states)

for i in range(0,len(qc.data)):
  print("gate#", i)
  for j in range (0, num_states):
    if Set_of_probability_ratios[i][j] != 0.0:
      Number_of_notes = Duration_in_quarter_notes * Set_of_probability_ratios[i][j]
      Note_Duration = 1/Set_of_probability_ratios[i][j]
      print(Number_of_notes)
      print(Note_Duration)
      index = 0
      while index < Number_of_notes:
        print("track", j, "pitch", List_of_MIDI_Notes[j], "time", 4*i + Note_Duration*index, "duration", Note_Duration) #For debugging
        Qubit_Superposition_Sonification_MIDI.addNote(track = j, channel = 0, pitch = List_of_MIDI_Notes[j], time = 4*i + Note_Duration*index, duration = Note_Duration, volume = 100)
        index += 1

with open("Qubit_Superposition_Sonification_MIDI.midi", 'wb') as output_file:
  Qubit_Superposition_Sonification_MIDI.writeFile(output_file)

files.download("Qubit_Superposition_Sonification_MIDI.midi")

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
gate# 0
4.0
1.0
track 0 pitch 48 time 0.0 duration 1.0
track 0 pitch 48 time 1.0 duration 1.0
track 0 pitch 48 time 2.0 duration 1.0
track 0 pitch 48 time 3.0 duration 1.0
gate# 1
4.0
1.0
track 0 pitch 48 time 4.0 duration 1.0
track 0 pitch 48 time 5.0 duration 1.0
track 0 pitch 48 time 6.0 duration 1.0
track 0 pitch 48 time 7.0 duration 1.0
4.0
1.0
track 2 pitch 74 time 4.0 duration 1.0
track 2 pitch 74 time 5.0 duration 1.0
track 2 pitch 74 time 6.0 duration 1.0
track 2 pitch 74 time 7.0 duration 1.0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>